# Pengerjaan Tugas Deep Learning Week 6 Menggunakan PyTorch

# Tahap Persiapan

## 1. Instalasi Modul yang diperlukan

In [ ]:
# !pip install numpy pandas matplotlib scikit-learn torch torchtext
#%pip install numpy pandas matplotlib scikit-learn torch torchtext


^C
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://download.pytorch.org/whl/nightly/cu128
Note: you may need to restart the kernel to use updated packages.


## 2. Mengimpor Library yang Dibutuhkan

In [2]:
# Import library yang diperlukan
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve, auc, confusion_matrix
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import torchtext
from torchtext.datasets import IMDB
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
import io
import re
from collections import Counter
import time

# Mengatur device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") # Menggunakan GPU jika tersedia, jika tidak menggunakan CPU
print(f"Menggunakan device: {device}")

OSError: [WinError 127] The specified procedure could not be found

## 3. Mendefinisikan Parameter dan Pre-processing Dataset

In [ ]:
# Konstanta dan parameter
NUM_WORDS = 40000
MAXLEN = 400
BATCH_SIZE = 64
EMBEDDING_DIM = 128
HIDDEN_DIM = 256
OUTPUT_DIM = 1
EPOCHS = 10
LEARNING_RATE = 0.001

# Memeriksa apakah GPU tersedia
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Menggunakan device: {DEVICE}")

# Langkah 1: Mengunduh dataset IMDB
print("Memulai pengunduhan dataset IMDb...")
train_iter, test_iter = IMDB(split=('train', 'test'))

# Langkah 2: Preprocessing data
# Fungsi untuk tokenisasi dan pembersihan teks
tokenizer = get_tokenizer("basic_english")

def yield_tokens(data_iter):
    for _, text in data_iter:
        text = re.sub(r'<.*?>', '', text.lower())  # Hapus tag HTML
        text = re.sub(r'[^\w\s]', '', text)        # Hapus tanda baca
        yield tokenizer(text)

# Langkah 3: Membangun kosakata
print("Membangun kosakata...")
train_iter, _ = IMDB(split=('train'))
vocab = build_vocab_from_iterator(yield_tokens(train_iter), 
                                    max_tokens=NUM_WORDS, 
                                    specials=["<unk>", "<pad>"])
vocab.set_default_index(vocab["<unk>"])
vocab_size = len(vocab)
print(f"Ukuran kosakata: {vocab_size}")

# Langkah 4: Mengubah teks menjadi indeks
def text_pipeline(text):
    text = re.sub(r'<.*?>', '', text.lower())  # Hapus tag HTML
    text = re.sub(r'[^\w\s]', '', text)        # Hapus tanda baca
    return [vocab[token] for token in tokenizer(text)]

def label_pipeline(label):
    return 1 if label == "pos" else 0

# Langkah 5: Mengubah dataset menjadi tensor
def process_data(data_iter, is_train=True):
    data = []
    labels = []
    
    for label, text in data_iter:
        processed_text = text_pipeline(text)
        # Padding atau truncating
        if len(processed_text) > MAXLEN:
            processed_text = processed_text[:MAXLEN]
        else:
            processed_text = processed_text + [vocab["<pad>"]] * (MAXLEN - len(processed_text))
        
        data.append(processed_text)
        labels.append(label_pipeline(label))
    
    # Konversi ke tensor
    data_tensor = torch.tensor(data, dtype=torch.long)
    labels_tensor = torch.tensor(labels, dtype=torch.float)
    
    if is_train:
        # Split data train menjadi train dan validation
        X_train, X_valid, y_train, y_valid = train_test_split(
            data_tensor, labels_tensor, test_size=0.2, random_state=42
        )
        return X_train, y_train, X_valid, y_valid
    else:
        return data_tensor, labels_tensor

# Memproses data train dan test
print("Memproses data train...")
train_iter, _ = IMDB(split=('train'))
X_train, y_train, X_valid, y_valid = process_data(train_iter)

print("Memproses data test...")
_, test_iter = IMDB(split=('test'))
X_test, y_test = process_data(test_iter, is_train=False)

# Menyiapkan dataloader
train_dataset = TensorDataset(X_train, y_train)
valid_dataset = TensorDataset(X_valid, y_valid)
test_dataset = TensorDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=BATCH_SIZE)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

## 4. Menyiapkan Fungsi Pelatihan dan Validasi

In [ ]:
# Fungsi untuk melatih model
def train_model(model, train_loader, valid_loader, optimizer, criterion, epochs):
    train_losses = []
    valid_losses = []
    train_accs = []
    valid_accs = []
    
    best_valid_loss = float('inf')
    
    print(f"Memulai pelatihan model {model.__class__.__name__}...")
    
    for epoch in range(epochs):
        start_time = time.time()
        
        # Training
        model.train()
        epoch_loss = 0
        epoch_acc = 0
        
        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = data.to(DEVICE), target.to(DEVICE)
            
            optimizer.zero_grad()
            prediction = model(data).squeeze(1)
            loss = criterion(prediction, target)
            
            loss.backward()
            optimizer.step()
            
            epoch_loss += loss.item()
            
            # Menghitung akurasi
            predicted_class = (prediction > 0.5).float()
            correct = (predicted_class == target).float().sum()
            accuracy = correct / len(target)
            epoch_acc += accuracy.item()
            
            if batch_idx % 100 == 0:
                print(f'Epoch: {epoch+1}/{epochs}, Batch: {batch_idx}/{len(train_loader)}, Loss: {loss.item():.4f}, Acc: {accuracy.item():.4f}')
        
        # Validasi
        model.eval()
        valid_loss = 0
        valid_acc = 0
        
        with torch.no_grad():
            for data, target in valid_loader:
                data, target = data.to(DEVICE), target.to(DEVICE)
                
                prediction = model(data).squeeze(1)
                loss = criterion(prediction, target)
                
                valid_loss += loss.item()
                
                predicted_class = (prediction > 0.5).float()
                correct = (predicted_class == target).float().sum()
                accuracy = correct / len(target)
                valid_acc += accuracy.item()
        
        # Menghitung metrik rata-rata
        train_loss = epoch_loss / len(train_loader)
        train_acc = epoch_acc / len(train_loader)
        valid_loss = valid_loss / len(valid_loader)
        valid_acc = valid_acc / len(valid_loader)
        
        # Menyimpan metrik untuk plotting
        train_losses.append(train_loss)
        valid_losses.append(valid_loss)
        train_accs.append(train_acc)
        valid_accs.append(valid_acc)
        
        end_time = time.time()
        epoch_mins, epoch_secs = divmod(end_time - start_time, 60)
        
        # Print hasil
        print(f'Epoch: {epoch+1}/{epochs} | Waktu: {epoch_mins:.0f}m {epoch_secs:.0f}s')
        print(f'\tTrain Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f}')
        print(f'\tValid Loss: {valid_loss:.4f} | Valid Acc: {valid_acc:.4f}')
        
        # Menyimpan model terbaik
        if valid_loss < best_valid_loss:
            best_valid_loss = valid_loss
            torch.save(model.state_dict(), f'{model.__class__.__name__}_model.pt')
            print(f'Model tersimpan dengan validasi loss: {valid_loss:.4f}')
    
    return {
        'train_losses': train_losses,
        'valid_losses': valid_losses,
        'train_accs': train_accs,
        'valid_accs': valid_accs
    }

## 5. Menyiapkan Fungsi Evaluasi

In [ ]:
# Fungsi untuk evaluasi model
def evaluate_model(model, test_loader, criterion):
    model.eval()
    
    all_predictions = []
    all_targets = []
    test_loss = 0
    
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            
            prediction = model(data).squeeze(1)
            loss = criterion(prediction, target)
            test_loss += loss.item()
            
            all_predictions.extend(prediction.cpu().numpy())
            all_targets.extend(target.cpu().numpy())
    
    # Konversi ke numpy array
    all_predictions = np.array(all_predictions)
    all_targets = np.array(all_targets)
    
    # Menghitung metrik evaluasi
    binary_preds = (all_predictions > 0.5).astype(int)
    
    accuracy = accuracy_score(all_targets, binary_preds)
    precision = precision_score(all_targets, binary_preds)
    recall = recall_score(all_targets, binary_preds)
    f1 = f1_score(all_targets, binary_preds)
    auc = roc_auc_score(all_targets, all_predictions)
    
    # Print hasil evaluasi
    print(f"Evaluasi Model {model.__class__.__name__}:")
    print(f"Test Loss: {test_loss/len(test_loader):.4f}")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print(f"AUC: {auc:.4f}")
    
    # Menghitung ROC curve
    fpr, tpr, _ = roc_curve(all_targets, all_predictions)
    
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'auc': auc,
        'fpr': fpr,
        'tpr': tpr,
        'test_loss': test_loss/len(test_loader)
    }

## 6. Menyiapkan Fungsi Visualisasi

In [ ]:
# Fungsi untuk membuat visualisasi hasil
def plot_training_history(history, model_name):
    plt.figure(figsize=(12, 5))
    
    # Plot Loss
    plt.subplot(1, 2, 1)
    plt.plot(history['train_losses'], label='Train Loss')
    plt.plot(history['valid_losses'], label='Valid Loss')
    plt.title(f'{model_name} - Loss History')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    
    # Plot Accuracy
    plt.subplot(1, 2, 2)
    plt.plot(history['train_accs'], label='Train Accuracy')
    plt.plot(history['valid_accs'], label='Valid Accuracy')
    plt.title(f'{model_name} - Accuracy History')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    
    plt.tight_layout()
    plt.savefig(f'training_history_pytorch_{model_name}.png')
    plt.show()

# Fungsi untuk membuat confusion matrix
def plot_confusion_matrix(y_true, y_pred, model_name):
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(8, 6))
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title(f'{model_name} - Confusion Matrix')
    plt.colorbar()
    tick_marks = np.arange(2)
    plt.xticks(tick_marks, ['Negative', 'Positive'])
    plt.yticks(tick_marks, ['Negative', 'Positive'])
    
    thresh = cm.max() / 2.
    for i, j in np.ndindex(cm.shape):
        plt.text(j, i, format(cm[i, j], 'd'),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")
    
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()
    plt.savefig(f'confusion_matrix_pytorch_{model_name}.png')
    plt.show()

# Fungsi untuk membuat ROC curve
def plot_roc_curve(results_dict):
    plt.figure(figsize=(10, 8))
    
    for model_name, result in results_dict.items():
        plt.plot(result['fpr'], result['tpr'], label=f'{model_name} (AUC = {result["auc"]:.4f})')
    
    plt.plot([0, 1], [0, 1], 'k--', label='Random')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve Comparison')
    plt.legend()
    plt.grid(True)
    plt.savefig('roc_curve_comparison_pytorch.png')
    plt.show()

# Fungsi untuk membuat bar chart perbandingan metrik
def plot_metrics_comparison(results_dict):
    metrics = ['accuracy', 'precision', 'recall', 'f1', 'auc']
    models = list(results_dict.keys())
    
    values = {metric: [results_dict[model][metric] for model in models] for metric in metrics}
    
    plt.figure(figsize=(12, 8))
    bar_width = 0.15
    index = np.arange(len(models))
    
    for i, metric in enumerate(metrics):
        plt.bar(index + i * bar_width, values[metric], bar_width, 
                label=metric.capitalize())
    
    plt.xlabel('Models')
    plt.ylabel('Scores')
    plt.title('Performance Metrics Comparison')
    plt.xticks(index + bar_width * 2, models)
    plt.legend()
    plt.grid(True, axis='y')
    plt.savefig('metrics_comparison_pytorch.png')
    plt.show()


# A. RNN dengan PyTorch

## 1. Menyusun Model RNN


In [ ]:
# Definisi Model RNN
class RNNModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, dropout=0.5):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=1)
        self.rnn1 = nn.RNN(embedding_dim, hidden_dim, dropout=dropout, batch_first=True)
        self.rnn2 = nn.RNN(hidden_dim, hidden_dim, dropout=dropout, batch_first=True)
        # Bidirectional RNN menghasilkan dua output untuk setiap langkah waktu
        # sehingga kita perlu mengalikan hidden_dim dengan 2
        # untuk mendapatkan ukuran yang benar untuk layer berikutnya
        self.fc1 = nn.Linear(hidden_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim // 2)
        self.fc3 = nn.Linear(hidden_dim // 2, output_dim)
        self.dropout = nn.Dropout(dropout)
        self.relu = nn.ReLU()
        
    def forward(self, text):
        # text: [batch size, sequence length]
        embedded = self.dropout(self.embedding(text))
        # embedded: [batch size, sequence length, embedding dim]
        
        output, hidden = self.rnn1(embedded)
        # output: [batch size, sequence length, hidden dim * 2]
        # hidden: [n layers * 2, batch size, hidden dim]

        output, hidden = self.rnn2(output)
        # output: [batch size, sequence length, hidden dim * 2]
        # hidden: [n layers * 2, batch size, hidden dim]
        
        # Menggunakan output dari langkah waktu terakhir
        hidden_concat = torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1)
        # hidden_concat: [batch size, hidden dim * 2]
        
        dense1 = self.relu(self.fc1(hidden_concat))
        dense1 = self.dropout(dense1)
        dense2 = self.relu(self.fc2(dense1))
        dense2 = self.dropout(dense2)
        
        # Output layer - menggunakan fungsi sigmoid untuk klasifikasi biner
        return torch.sigmoid(self.fc3(dense2))

# Inisialisasi model
model_rnn = RNNModel(
        vocab_size=vocab_size,
        embedding_dim=EMBEDDING_DIM,
        hidden_dim=HIDDEN_DIM,
        output_dim=OUTPUT_DIM
    ).to(DEVICE)

## 2. Melatih Model

In [ ]:
# Loss function dan optimizer
criterion = nn.BCELoss()
rnn_optimizer = optim.Adam(model_rnn.parameters(), lr=LEARNING_RATE)

# Melatih model RNN
print("Melatih model RNN...")
rnn_history = train_model(model_rnn, train_loader, valid_loader, rnn_optimizer, criterion, EPOCHS)

## 3. Mengevaluasi Model dengan Menghitung Akurasi, Presisi, Recall, F1Squared, ROC, dan AUC-ROC

In [ ]:
# Evaluasi model RNN
rnn_results = evaluate_model(model_rnn, test_loader, criterion)

## 4. Memvisualisasikan Prediksi dari model yang telah dilatih

In [ ]:
# Plotting hasil pelatihan
plot_training_history(rnn_history, "RNN")

# Plotting confusion matrix
y_pred_rnn = (rnn_results['all_predictions'] > 0.5).astype(int)
plot_confusion_matrix(y_test, y_pred_rnn, "RNN")

## 5. Menyimpan Model

In [ ]:
# Menyimpan model
torch.save(model_rnn.state_dict(), 'model_pytorch_rnn.pt')
print("Model RNN disimpan sebagai 'model_pytorch_rnn.pt'")

# B. LSTM dengan PyTorch

## 1. Menyusun Model LSTM


In [ ]:
# Definisi Model LSTM
class LSTMModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, dropout=0.5):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=1)
        self.lstm1 = nn.LSTM(embedding_dim, hidden_dim, dropout=dropout, batch_first=True)
        self.lstm2 = nn.LSTM(hidden_dim, hidden_dim, dropout=dropout, batch_first=True)
        # Bidirectional LSTM menghasilkan dua output untuk setiap langkah waktu
        # sehingga kita perlu mengalikan hidden_dim dengan 2
        # untuk mendapatkan ukuran yang benar untuk layer berikutnya
        self.fc1 = nn.Linear(hidden_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim // 2)
        self.fc3 = nn.Linear(hidden_dim // 2, output_dim)
        self.dropout = nn.Dropout(dropout)
        self.relu = nn.ReLU()
        
    def forward(self, text):
        # text: [batch size, sentence length]
        embedded = self.dropout(self.embedding(text))
        # embedded: [batch size, sentence length, embedding dim]
        
        output, (hidden, cell) = self.lstm1(embedded)
        # output: [batch size, sentence length, hidden dim * 2]
        # hidden: [n layers * 2, batch size, hidden dim]
        # cell: [n layers * 2, batch size, hidden dim]

        output, (hidden, cell) = self.lstm2(output)
        # output: [batch size, sentence length, hidden dim * 2]
        
        # Menggunakan output dari langkah waktu terakhir
        hidden_concat = torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1)
        # hidden_concat: [batch size, hidden dim * 2]
        
        dense1 = self.relu(self.fc1(hidden_concat))
        dense1 = self.dropout(dense1)
        dense2 = self.relu(self.fc2(dense1))
        dense2 = self.dropout(dense2)
        
        # Output layer
        return torch.sigmoid(self.fc3(dense2))
    
model_lstm = LSTMModel(
        vocab_size=vocab_size,
        embedding_dim=EMBEDDING_DIM,
        hidden_dim=HIDDEN_DIM,
        output_dim=OUTPUT_DIM
    ).to(DEVICE)

## 2. Melatih Model

In [ ]:
# Inisialisasi loss function dan optimizer
criterion = nn.BCELoss()
lstm_optimizer = optim.Adam(model_lstm.parameters(), lr=LEARNING_RATE)

# Melatih model LSTM
print("Melatih model LSTM...")
lstm_history = train_model(model_lstm, train_loader, valid_loader, lstm_optimizer, criterion, EPOCHS)

## 3. Mengevaluasi Model dengan Menghitung Akurasi, Presisi, Recall, F1Squared, ROC, dan AUC-ROC

In [ ]:
# Evaluasi model LSTM
lstm_results = evaluate_model(model_lstm, test_loader, criterion)

## 4. Memvisualisasikan Prediksi dari model yang telah dilatih

In [ ]:
# Plotting hasil pelatihan
plot_training_history(lstm_history, "LSTM")

# Plotting confusion matrix
y_pred_lstm = (lstm_results['all_predictions'] > 0.5).astype(int)
plot_confusion_matrix(y_test, y_pred_lstm, "LSTM")

## 5. Menyimpan Model

In [ ]:
# Menyimpan model
torch.save(model_lstm.state_dict(), 'model_pytorch_lstm.pt')
print("Model LSTM disimpan sebagai 'model_pytorch_lstm.pt'")

# C. GRU dengan PyTorch

## 1. Menyusun Model GRU


In [ ]:
# Definisi Model GRU
class GRUModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, dropout=0.5):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=1)
        self.gru1 = nn.GRU(embedding_dim, hidden_dim, dropout=dropout, batch_first=True)
        self.gru2 = nn.GRU(hidden_dim, hidden_dim, dropout=dropout, batch_first=True)
        self.fc1 = nn.Linear(hidden_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim // 2)
        self.fc3 = nn.Linear(hidden_dim // 2, output_dim)
        self.dropout = nn.Dropout(dropout)
        self.relu = nn.ReLU()
        
    def forward(self, text):
        # text: [batch size, sentence length]
        embedded = self.dropout(self.embedding(text))
        # embedded: [batch size, sentence length, embedding dim]
        
        output, hidden = self.gru1(embedded)
        # output: [batch size, sentence length, hidden dim * 2]
        # hidden: [n layers * 2, batch size, hidden dim]

        output, hidden = self.gru2(output)
        # output: [batch size, sentence length, hidden dim * 2]
        
        # Menggunakan output dari langkah waktu terakhir
        hidden_concat = torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1)
        # hidden_concat: [batch size, hidden dim * 2]
        
        dense1 = self.relu(self.fc1(hidden_concat))
        dense1 = self.dropout(dense1)
        dense2 = self.relu(self.fc2(dense1))
        dense2 = self.dropout(dense2)
        
        # Output layer
        return torch.sigmoid(self.fc3(dense2))
    
print("\n===== Model GRU =====")
model_gru = GRUModel(
        vocab_size=vocab_size,
        embedding_dim=EMBEDDING_DIM,
        hidden_dim=HIDDEN_DIM,
        output_dim=OUTPUT_DIM
    ).to(DEVICE)

## 2. Melatih Model

In [ ]:
# Inisialisasi loss function dan optimizer
criterion = nn.BCELoss()
gru_optimizer = optim.Adam(model_gru.parameters(), lr=LEARNING_RATE)

# Melatih model GRU
print("Melatih model gru...")
gru_history = train_model(model_gru, train_loader, valid_loader, gru_optimizer, criterion, EPOCHS)

## 3. Mengevaluasi Model dengan Menghitung Akurasi, Presisi, Recall, F1Squared, ROC, dan AUC-ROC

In [ ]:
# Evaluasi model GRU
print("Evaluasi model GRU...")
gru_results = evaluate_model(model_gru, test_loader, criterion)

## 4. Memvisualisasikan Prediksi dari model yang telah dilatih

In [ ]:
# Plotting hasil pelatihan
plot_training_history(gru_history, "GRU")

# Plotting confusion matrix
y_pred_gru = (gru_results['all_predictions'] > 0.5).astype(int)
plot_confusion_matrix(y_test, y_pred_gru, "GRU")

## 5. Menyimpan Model

In [ ]:
# Menyimpan model
torch.save(model_gru.state_dict(), 'model_pytorch_gru.pt')
print("Model GRU disimpan sebagai 'model_pytorch_gru.pt'")

# Perbandingan antara model

In [ ]:
# Menghitung perbandingan untuk semua model
results_dict = {
    'RNN': rnn_results,
    'LSTM': lstm_results,
    'GRU': gru_results
}

# Plotting ROC curve
plot_roc_curve(results_dict)
# Plotting perbandingan metrik
plot_metrics_comparison(results_dict)

# Perbandingan antara model 

## 1. Arsitektur Model

### MLP (Multi-Layer Perceptron):

MLP menggunakan lapisan fully connected (dense) untuk memproses data.
Input gambar 32x32x3 diratakan menjadi vektor 1D (3072 fitur), sehingga kehilangan informasi spasial (lokasi piksel).
MLP lebih cocok untuk data tabular atau data yang tidak memiliki struktur spasial.

### CNN (Convolutional Neural Network):

CNN menggunakan lapisan konvolusi untuk menangkap pola lokal (seperti tepi, tekstur) dalam gambar.
Mempertahankan informasi spasial melalui operasi konvolusi dan pooling.
CNN lebih cocok untuk data citra karena mampu mengenali pola visual dengan lebih baik.

## 2. Kemampuan Generalisasi

MLP: Karena kehilangan informasi spasial, MLP cenderung memiliki performa lebih rendah pada data citra.
Membutuhkan lebih banyak parameter untuk memproses data citra, sehingga lebih rentan terhadap overfitting.

CNN: CNN lebih efisien dalam memproses data citra karena memanfaatkan pola lokal.
Dengan jumlah parameter yang lebih sedikit dibandingkan MLP, CNN memiliki kemampuan generalisasi yang lebih baik.

## 3. Hasil Evaluasi
CNN biasanya menghasilkan akurasi, presisi, recall, F1-score, dan AUC-ROC yang lebih tinggi dibandingkan MLP pada dataset citra seperti CIFAR-10.
Dari file ini, CNN rata-rata menunjukkan performa yang lebih baik dalam menangkap pola visual dibandingkan MLP (seperti contoh, Akurasi model CNN mendekati 90%, dibandingkan dengan model MLP yang hanya mencapai sekitar 60%).

## 4. Kompleksitas Komputasi

MLP: Lebih sederhana dalam implementasi, tetapi membutuhkan lebih banyak memori karena semua neuron terhubung penuh.
Tidak memanfaatkan GPU secara optimal.

CNN: Lebih kompleks karena melibatkan operasi konvolusi, pooling, dan normalisasi.
Memanfaatkan GPU dengan lebih baik, sehingga lebih efisien untuk data citra.


## 5. Kesimpulan
CNN adalah pilihan yang lebih baik untuk prediksi citra pada dataset CIFAR-10 karena mampu menangkap pola visual dengan lebih baik, menghasilkan performa yang lebih tinggi, dan lebih efisien dalam memproses data citra dibandingkan MLP.

# Penjelasan Persamaan Matematika dalam Model MLP dan CNN

## 1. Konvolusi (Convolution)
Dalam model CNN, operasi konvolusi adalah operasi dasar yang menjadi inti dari arsitektur. Secara matematis, konvolusi 2D dapat didefinisikan sebagai:

$$(I * K)(i, j) = \sum_{m} \sum_{n} I(i-m, j-n) \cdot K(m, n)$$

dimana:
- $I$ adalah citra input
- $K$ adalah kernel atau filter
- $*$ adalah operator konvolusi
- $i, j$ adalah indeks piksel output
- $m, n$ adalah indeks kernel

Konvolusi bekerja dengan menggeser kernel di atas citra input, mengalikan nilai-nilai yang bertumpuk, dan menjumlahkan hasil untuk menghasilkan piksel output.

## 2. Aktivasi ReLU (Rectified Linear Unit)

Fungsi aktivasi ReLU yang digunakan dalam model kita didefinisikan sebagai:

$$f(x) = \max(0, x)$$

ReLU akan mengubah semua nilai negatif menjadi 0, sementara nilai positif tetap tidak berubah. Ini membantu model untuk belajar fungsi non-linear dan mengatasi masalah vanishing gradient.

## 3. Batch Normalization

Batch Normalization digunakan untuk menstabilkan dan mempercepat proses pembelajaran dengan menormalkan aktivasi di setiap lapisan. Rumus matematisnya adalah:

$$\hat{x}_i = \frac{x_i - \mu_B}{\sqrt{\sigma_B^2 + \epsilon}}​$$

$$y_i = \gamma \cdot \hat{x}_i + \beta$$

dimana:
- $x_i$ adalah input ke batch normalization
- $\mu_B$ adalah rata-rata mini-batch
- $\sigma_B^2$ adalah varians mini-batch
- $\epsilon$ adalah konstanta kecil untuk stabilitas numerik
- $\gamma$ dan $\beta$ adalah parameter yang dipelajari
- $y_i$ adalah output batch normalization

## 4. Max Pooling

Max Pooling digunakan untuk mengurangi dimensi spasial dengan memilih nilai maksimum dari setiap region. Secara matematis:

$$\text{MaxPool}(X)_{i,j} = \max_{m,n \in R_{i,j}} X_{m,n}​$$

dimana $R_{i,j}$ adalah region lokal yang berpusat di lokasi $(i,j)$.

## 5. Dropout

Dropout adalah teknik regularisasi yang mencegah overfitting dengan secara acak mematikan neuron selama training. Secara matematis, untuk setiap neuron:

$$\begin{cases}
x/p & \text{dengan probabilitas } p \\
0 & \text{dengan probabilitas } 1-p
\end{cases}$$

dimana $p$ adalah probabilitas mempertahankan (keep probability) dan $1-p$ adalah dropout rate.

## 6. Forward Pass dalam MLP
Dalam MLP, forward pass untuk lapisan fully-connected didefinisikan sebagai:

$$z^{[l]} = W^{[l]} \cdot a^{[l-1]} + b^{[l]}$$
$$a^{[l]} = g(z^{[l]})$$

dimana:
- $W^{[l]}$ adalah matriks bobot untuk lapisan ke-l
- $a^{[l-1]}$ adalah aktivasi dari lapisan sebelumnya
- $b^{[l]}$ adalah vektor bias
- $g$ adalah fungsi aktivasi (seperti ReLU)
- $z^{[l]}$ adalah input terbobot
- $a^{[l]}$ adalah output lapisan

## 7. Fungsi Loss Cross-Entropy
Cross-entropy loss yang digunakan untuk klasifikasi multi-kelas didefinisikan sebagai:

$$L = -\frac{1}{N} \sum_{i=1}^{N} \sum_{c=1}^{C} y_{i,c} \log(p_{i,c})$$

dimana:
- $N$ adalah jumlah sampel
- $C$ adalah jumlah kelas
- $y_{i,c}$ adalah indikator (0 atau 1) apakah kelas c adalah label yang benar untuk sampel i
- $p_{i,c}$ adalah probabilitas yang diprediksi bahwa sampel i adalah kelas c

## 8. Optimizer Adam
Adam (Adaptive Moment Estimation) adalah algoritma optimasi yang menggabungkan RMSprop dan momentum. Adam memperbarui bobot menggunakan rumus:

$$m_t = \beta_1 \cdot m_{t-1} + (1 - \beta_1) \cdot g_t$$
$$v_t = \beta_2 \cdot v_{t-1} + (1 - \beta_2) \cdot g_t^2$$
$$\hat{m}_t = \frac{m_t}{1 - \beta_1^t}$$
$$\hat{v}_t = \frac{v_t}{1 - \beta_2^t}$$
$$\theta_t = \theta_{t-1} - \alpha \cdot \frac{\hat{m}_t}{\sqrt{\hat{v}_t} + \epsilon}$$

dimana:
- $g_t$ adalah gradien pada waktu t
- $m_t$ dan $v_t$ adalah estimasi momen pertama dan kedua
- $\hat{m}_t$ dan $\hat{v}_t$ adalah estimasi momen yang dikoreksi
- $\alpha$ adalah learning rate
- $\beta_1$ dan $\beta_2$ adalah exponential decay rates
- $\epsilon$ adalah konstanta kecil untuk stabilitas numerik
- $\theta_t$ adalah parameter yang diperbarui

## 9. Softmax
Fungsi softmax digunakan pada output layer untuk mengubah logits menjadi probabilitas yang berjumlah 1:

$$\text{softmax}(x)_i = \frac{e^{x_i}}{\sum_{j=1}^{k} e^{x_j}}$$

dimana $x_i$ adalah nilai logit untuk kelas i dan k adalah jumlah kelas.

# Penjelasan Metrik Evaluasi:
## Akurasi

$$ \text{Accuracy} = \frac{\text{True Positives} + \text{True Negatives}}{\text{Total Samples}} $$

Menunjukkan seberapa sering model memprediksi dengan benar.


## Presisi

$$ \text{Precision} = \frac{\text{True Positives}}{\text{True Negatives} + \text{False Positives}} $$

Seberapa andal prediksi positif model.


## Recall (Sensitivity)

$$ \text{Recall} = \frac{\text{True Positives}}{\text{True Positives} + \text{False Negatives}} $$

Seberapa baik model menangkap kelas positif.


## F1-Score

$$ \text{F1-Score} = 2 \times \frac{\text{Precision} \times \text{Recall}}{\text{Precision} + \text{Recall}} $$

Rata-rata harmonik antara presisi dan recall.


## ROC (Receiver Operating Characteristic) Curve

ROC Curve memplot hubungan antara True Positive Rate (TPR) dan False Positive Rate (FPR):

$$ \text{True Positive Rate (TPR)} = \text{Recall} $$

$$ \text{False Positive Rate (FPR)} = \frac{\text{False Positives}}{ \text{False Positives} + \text{True Negatives}} $$


## AUC-ROC

$$ AUC = \int_{0}^{1} TPR(x) \, dx $$

AUC (Area Under Curve) adalah area di bawah kurva ROC (Receiver Operating Characteristic)